# Failed jobs - map samples and reset job

In [13]:
# %pip install awswrangler

In [14]:
import json
import boto3 
import numpy as np
import pandas as pd
import awswrangler as wr

from sagemaker import get_execution_role
role = get_execution_role()

In [33]:
# reading file contents from s3 bucket
bucket = 'ons-dl-dev-landing'
prefix = 'ons/mpo_ds_nlp/regioes-newjob/cosr-nco/'
obj_uri = 's3://{}/{}'.format(bucket, prefix)

cl = boto3.client('s3')
contents = cl.list_objects(Bucket=bucket, Prefix=prefix)['Contents']

file_list = []
for file in contents:
    file_list.append(file['Key'])

In [34]:
#file_list

In [35]:
# selecting just consolidated-annotations
file_list = [i for i in file_list if 'consolidated-annotation' in i]
# file_list

In [36]:
# reading output from ground truth entity recognition job
ref_dict = {
    'file': [],
    'sample_id':[],
    'start':[],
    'end':[],
    'label':[]
}

for file in file_list: 
    obj_uri = 's3://{}/{}'.format(bucket,file)
    json_obj = pd.read_json(obj_uri)
    try:
        metadata, region = list(json_obj['consolidatedAnnotation'][0]['content'].keys())
        for entity in json_obj['consolidatedAnnotation'][0]['content'][region]['annotations']['entities']:
            ref_dict['file'].append(file)
            ref_dict['start'].append(entity['startOffset'])
            ref_dict['end'].append(entity['endOffset'])
            ref_dict['label'].append(entity['label'])
            ref_dict['sample_id'].append(json_obj['datasetObjectId'][0])
    except:
        for entity in json.loads(json_obj['annotations'][0][0]['annotationData']['content'])['crowd-entity-annotation']['entities']:
            ref_dict['file'].append(file)
            ref_dict['start'].append(entity['startOffset'])
            ref_dict['end'].append(entity['endOffset'])
            ref_dict['label'].append(entity['label'])
            ref_dict['sample_id'].append(json_obj['datasetObjectId'][0])
        
ref_df = pd.DataFrame(ref_dict)
ref_df[['sample_id','start','end']] = ref_df[['sample_id','start','end']].astype(int)
ref_df

,file,sample_id,start,end,label
0,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,368,298,305,VALOR_COM_UNID.MEDIDA
1,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,368,309,319,SUBESTACAO
2,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,368,669,697,EQUIPAMENTO
3,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,368,1012,1069,ACAO_A_EXECUTAR
4,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,368,1215,1266,ESTADO_OPERATIVO
...,...,...,...,...,...
4253,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,256,776,777,OPERADOR_MATEMATICO
4254,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,256,277,283,VALOR_COM_UNID.MEDIDA
4255,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,256,778,784,VALOR_COM_UNID.MEDIDA
4256,ons/mpo_ds_nlp/regioes-newjob/cosr-nco/COSR-NC...,256,485,496,ESTADO_OPERATIVO


In [53]:
# set(ref_df['sample_id'])

In [52]:
obj_uri = 's3://{}/{}'.format(bucket,'ons/mpo_ds_nlp/regioes-newjob/cosr-nco/samples.csv')
samples_df = pd.read_csv(obj_uri)
extra_sample = pd.DataFrame(columns=['samples'])
extra_sample['samples'] = [samples_df.columns[0]]
samples_df.columns = ['samples']
samples_df = pd.concat([extra_sample, samples_df], axis=0)
samples_df.reset_index(inplace=True,drop=True)

remaining_samples = samples_df.drop(set(ref_df['sample_id']))
remaining_samples

,samples
0,3.1.1. Desenergização da LT 230 kV Tucur...
1,3.2.1. Desenergização da LT 230 kV Altam...
2,3.3.1. Desenergização da LT 230 kV Altam...
3,3.4.1. Desenergização da LT 230 kV Trans...
4,3.5.1. Desenergização da LT 230 kV Tapaj...
5,3.6.1. Desenergização da LT 230 kV Altam...
6,4.10.1. Desenergização do Autotransformador ...
7,4.2.1. Desenergização do Autotransformad...
8,4.4.1. Desenergização do Transformador T...
9,4.7.1. Desenergização do Autotransformad...


In [54]:
# bucket = 'ons-dl-dev-landing'
# prefix = 'ons/mpo_ds_nlp/regioes-newjob/cosr-nco-debug-1/samples.csv'
# output_uri = 's3://{}/{}'.format(bucket, prefix)
# remaining_samples.to_csv(output_uri, header=False, index=False)